In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import re
import time
import os
import json
from PIL import Image 
import PIL 
pd.set_option('display.max_columns', None)
import random
import codecs
import subprocess

# Headers

In [2]:
headers = {
    'authority': 'www.paruvendu.fr',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7'
}

# Save pages x15

In [3]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "paruvendu" + "-" + datetime_1 + "-" + str(page)
    with open("pages/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [4]:
pages = np.arange(1, 3)
pages

array([1, 2])

In [5]:
for page_ in pages:
    req_x15 = requests.get(f"https://www.paruvendu.fr/auto-moto/listefo/default/default?moto-typeRech=&r=VMOMO000&px1=ex:%2050000&r2=&codeINSEE=&lo=&pa=&ray=100&cy=&nrj=&km1=&a0=&fulltext=&p={page_}",
                           headers = headers)
    save_page_list(req_x15, page=page_)
    time.sleep(random.randint(4, 5))
    print("x15 saved: " + str(page_))

x15 saved: 1
x15 saved: 2


# Extract data from saved x15

## Test file

In [7]:
file_test_name = [file for file in os.listdir("pages") if file.endswith("html")][1]
file_test_name

'paruvendu-2021-06-02_21h20-30.html'

In [8]:
file_check_x15_good = f"pages/{file_test_name}"
file_check_x15_good

'pages/paruvendu-2021-06-02_21h20-30.html'

## Get prices from saved x15

In [9]:
def get_prices_from_saved_x15(r):
    
    # set empty price list
    price_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("div", class_="ergov3-priceannonce")
            if len(step2)>0:
                step3 = re.findall('[0-9]', step2[0].text)
                if len(step3)>0:
                    price = float("".join(step3))
                    price_list.append(price)
                else:
                    price_list.append(np.nan)
            else:
                return None
    else:
        return None
                    
    return price_list

In [10]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    prices = get_prices_from_saved_x15(readable_html)
prices

[5000.0,
 6200.0,
 3200.0,
 17500.0,
 3790.0,
 2000.0,
 4500.0,
 4500.0,
 13500.0,
 3500.0,
 16500.0,
 6499.0,
 6200.0,
 8500.0,
 10250.0]

## Get references from saved x15

for this website there is no Reference on page x15 but we can still find a unique ID that will differ from announce reference

In [11]:
def get_unique_ID_from_saved_x15(r):
    
    # set empty uniq_ID list
    uniq_ID_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract uniq ID
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            if "data-id" in k.attrs:
                uniq_ID_list.append(k.attrs["data-id"])
            else:
                uniq_ID_list.append(np.nan)
    else:
        return None
    
    return uniq_ID_list

In [12]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    unique_id = get_unique_ID_from_saved_x15(readable_html)
unique_id

['1253979582',
 '1253978698',
 '1253977270',
 '1253977145',
 '1253976914',
 '1253889260',
 '1253917685',
 '1253962187',
 '1253870175',
 '1253973918',
 '1253971839',
 '1246648333',
 '1253894110',
 '1253963695',
 '1250913038']

## Get urls from saved x15

In [13]:
def get_urls_from_saved_x15(r):
    
    # set empty url list
    url_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("a")
            if len(step2)>2:
                if "href" in step2[1].attrs:
                    url_list.append(step2[1].attrs["href"])
                else:
                    url_list.append(np.nan)
            else:
                return None
    else:
        return None
    
    return url_list

In [14]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    urls_list = get_urls_from_saved_x15(readable_html)
urls_list

['https://www.paruvendu.fr/a/moto-scooter/moto/bmw/1200-cm3/1253979582A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/689-cm3/1253978698A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/suzuki/650-cm3/1253977270A1KVMOMOSU',
 'https://www.paruvendu.fr/a/moto-scooter/moto/harley-davidson/1680-cm3/1253977145A1KVMOMOHD',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/900-cm3/1253976914A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/kawasaki/1000-cm3/1253889260A1KVMOMOKA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/royal-enfield/535-cm3/1253917685A1KVMOMORE',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/1300-cm3/1253962187A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/harley-davidson/1690-cm3/1253870175A1KVMOMOHD',
 'https://www.paruvendu.fr/a/moto-scooter/moto/honda/900-cm3/1253973918A1KVMOMOHO',
 'https://www.paruvendu.fr/a/moto-scooter/moto/harley-davidson/1690-cm3/1253971839A1KVMOMOHD',
 'https://www.paruvendu.fr/

## Get title from saved x15

In [15]:
def get_titles_from_saved_x15(r):
    
    # set empty title list
    title_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract title
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("h3")
            if len(step2)>0:
                title_list.append(step2[0].text.replace("\n", "").replace("\t", "").replace("Moto ", "").strip())
            else:
                title_list.append(np.nan)
    else:
        return None
    #
    return title_list

In [16]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    title_list = get_titles_from_saved_x15(readable_html)
title_list

['BMW',
 'YAMAHA',
 'SUZUKI',
 'HARLEY-DAVIDSON',
 'YAMAHA',
 'KAWASAKI',
 'ROYAL ENFIELD',
 'YAMAHA',
 'HARLEY-DAVIDSON',
 'HONDA',
 'HARLEY-DAVIDSON',
 'TRIUMPH',
 'KAWASAKI',
 'SUZUKI',
 'DUCATI 1198 s']

## temp df template

In [17]:
temp_template = pd.DataFrame({"url": [np.nan],
                              "unique id": [np.nan],
                              "price": [np.nan],
                              "title": [np.nan]
                            })
temp_template

,url,unique id,price,title
0,NaN,NaN,NaN,NaN


In [18]:
def save_temporary_data(url_, uniq_id_, price_, title_):

    # load csv if exists or starting from template
    data_paruvendu = "../../../tresboncoin/data/paruvendu_ad.csv"
    #
    if os.path.isfile(data_paruvendu) is False:
        df = temp_template.copy()
    else:
        data = pd.read_csv(data_paruvendu)
        df = temp_template.copy()
        
    # adding data
    df["url"] = url_
    df["unique id"] = uniq_id_
    df["price"] = price_
    df["title"] = title_
    
    # concatenate to csv and write
    try:
        data = pd.concat([data, df], axis=0)
        data.to_csv(path_or_buf = data_paruvendu, index=False)
    except:
        df.to_csv(path_or_buf = data_paruvendu, index=False)
        
    return

# Download single announce pages

##### check if an announce with different price but same unique id exists in dataframe.

In [19]:
data_paruvendu = "../../../tresboncoin/data/paruvendu.csv"

In [20]:
data_price = pd.read_csv(data_paruvendu)

In [21]:
data_price.shape

(1410, 20)

In [22]:
def add_the_announce(df, uniq_id, price):
    if int(uniq_id) in list(df["unique id"]):
        index_ = df.index[df['unique id'] == int(uniq_id)].tolist()[0]
        
        # if an announce with same uniq id and same price is found, return true and skip
        return df.iloc[index_]["price"]!=price
    
    # else, return false and add the announce
    return True

##### save annonce function

In [23]:
def save_page_uniq(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_name = "paruvendu" + "-" + uniq_id + "-" + datetime_1
    with open("annonces/" + page_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

##### check if id exists or price changed and save

In [ ]:
page_count = 1
for html_file in [file for file in os.listdir("pages") if file.endswith("html")]:
    with open(f"pages/{html_file}", 'r') as f:
        readable_html = f.read()
        urls_list = get_urls_from_saved_x15(readable_html)
        unique_id = get_unique_ID_from_saved_x15(readable_html)
        prices_list = get_prices_from_saved_x15(readable_html)
    if urls_list != None:
        for url_single, uniq_id, price_ in zip(urls_list, unique_id, prices_list):
            if add_the_announce(data_price, uniq_id, price_):
                req_uniq = requests.get(url_single, headers = headers)
                save_page_uniq(req_uniq, uniq_id)
                print(str(page_count) + ". saved page id: " + uniq_id)
                time.sleep(1)
                page_count += 1
            else:
                print("NO! " + url_single)

In [25]:
1253977270 in list(data_price["unique id"])

True

In [26]:
data_price.iloc[1158]["url"]

'https://www.paruvendu.fr/a/moto-scooter/moto/honda/600-cm3/1254313672A1KVMOMOHO'

# Process single announces
Saving scaled images x3 and adding announce to DataFrame

## DataFrame template

In [27]:
announce_template = pd.DataFrame({"url": [np.nan],
                                  "reference": [np.nan],
                                  "unique id": [np.nan],
                                  "date_scrapped": [np.nan],
                                  "announce_publication_date": [np.nan],
                                  "vehicle brand": [np.nan],
                                  "vehicle type": [np.nan],
                                  "moto scoot": [np.nan],
                                  "color": [np.nan],
                                  "vehicle condition": [np.nan],
                                  "price": [np.nan],
                                  "city": [np.nan],
                                  "postal code": [np.nan],
                                  "vehicle release date": [np.nan],
                                  "mileage": [np.nan],
                                  "Fiscal power [HP]": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "comments": [np.nan],
                                  "seller": [np.nan],
                                  "seller_name": [np.nan]})
announce_template

,url,reference,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,moto scoot,color,vehicle condition,price,city,postal code,vehicle release date,mileage,Fiscal power [HP],engine capacity [CC],comments,seller,seller_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## test announce

In [32]:
file_check_name_single = "paruvendu-1254634230-2021-06-02_21h41"
#
file_check_single = f"annonces/{file_check_name_single}.html"
file_check_single

'annonces/paruvendu-1254634230-2021-06-02_21h41.html'

## Functions

### Get url

In [33]:
def get_url(r):
    
    # set empty url list
    url_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('meta[property*="og:url"]')
    if len(step1)>0:
        return step1[0].attrs["content"].split("?")[0]
    else:
        return None

In [34]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
g_url

'https://www.paruvendu.fr/a/moto-scooter/moto/triumph/1215-cm3/1254634230A1KVMOMOTR'

### Get reference

In [35]:
def get_reference(r):
    
    # set empty reference list
    ref_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract ref
    step1 = soup.select('div[class*="vvdetails14_refdate"]')
    if len(step1)>0:
        return step1[0].text.replace("\n", "").replace("\t", "").split("ParuVendu")[-1].split("-")[0].strip()
    else:
        return None

In [36]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_ref = get_reference(readable_html)
g_ref

'WV167689756'

### Get unique ID

In [37]:
def get_uniq_id(url_):
    return url_.split("/")[-1].split("A1")[0]

In [38]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
get_uniq_id(g_url)

'1254634230'

### get publication date

In [39]:
def get_publication_date(r):
    
    # set empty 
    pub_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="vvdetails14_refdate"]')
    if len(step1)>0:
        try:
            return step1[0].text.replace("\n", "").replace("\t", "").split("ParuVendu")[-1].split("-")[1].strip().split(" ")[1]
        except:
            return np.nan
    else:
        return None

In [40]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_pub = get_publication_date(readable_html)
g_pub

'02/06/2021'

### Get vehicule brand

In [87]:
def get_brand(r):
    
    # set empty 
    brand_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h1")[0].text.replace(u'\xa0', u' ').strip().split(" ")[-1]
    else:
        return None

In [88]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_brand = get_brand(readable_html)
g_brand

'TRIUMPH'

### Get vehicule type

In [43]:
def get_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Type") >= 0:
                try:
                    return k.select("span")[0].text.replace("\n", "")
                except:
                    return np.nan
    return None

In [44]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_type(readable_html)
g_type

'Trail'

### get moto scoot

In [85]:
def get_moto(r):
        
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h1")[0].text.replace(u'\xa0', u' ').strip().split(" ")[0]
    else:
        return None

In [86]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_moto = get_moto(readable_html)
g_moto

'MOTO'

### get color

In [45]:
def get_color(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Coul") >= 0:
                try:
                    return k.select("span")[0].text.replace("\n", "")
                except:
                    return np.nan
    return None

In [46]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_color = get_color(readable_html)
g_color

'Rouge'

### get vehicle condition

In [47]:
def get_cond(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Etat du") >= 0:
                try:
                    return k.select("span")[0].text.replace("\n", "")
                except:
                    return np.nan
    return None

In [48]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_cond = get_cond(readable_html)
g_cond

'Irréprochable'

### get fiscal power

In [49]:
def get_power(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.replace("\n", " ").find("Puiss") >= 0:
                try:
                    return int(k.select("span")[0].text.strip())
                except:
                    return np.nan
    return None

In [50]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_power = get_power(readable_html)
g_power

11

### get price

In [69]:
def get_price(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Pri") >= 0:
                try:
                    return float("".join(re.findall(r'\d+', k.select("span")[0].text.replace("\n", ""))))
                except:
                    return np.nan
    return None

In [70]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_price = get_price(readable_html)
g_price

12900.0

### get city

In [71]:
def get_city(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h2")[0].text.split(" ")[-1].replace("\n", "")
    return None

In [72]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_city = get_city(readable_html)
g_city

'Lyon'

### get postal code

In [73]:
def get_postalcode(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h2")[0].text.split(" ")[0].replace("\n", "")
    return None

In [74]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_post = get_postalcode(readable_html)
g_post

'69004'

### get vehicle release date

In [75]:
def get_releasedate(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Année") >= 0:
                try:
                    return int("".join(re.findall(r'\d+', k.select("span")[0].text.replace("\n", ""))))
                except:
                    return np.nan
    return None

In [76]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_release = get_releasedate(readable_html)
g_release

2018

### get vehicle mileage

In [77]:
def get_mileage(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Kilom") >= 0:
                try:
                    return int("".join(re.findall(r'\d+', k.select("span")[0].text.replace("\n", ""))))
                except:
                    return np.nan
    return None

In [78]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_mileage = get_mileage(readable_html)
g_mileage

18264

### get engine capacity

In [79]:
def get_capa(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li')
        for k in step2:
            if k.text.find("Cyl") >= 0:
                try:
                    return int("".join(re.findall(r'\d+', k.select("span")[0].text.replace("\n", ""))))
                except:
                    return np.nan
    return None

In [80]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_capa = get_capa(readable_html)
g_capa

1215

### get comment

In [81]:
def get_comment(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('div[class*="txt_annonceauto"]')
        if len(step2)>0:
            raw_text = step2[0].text.split("Prix")[0].strip()
            #raw_text = unicode(raw_text, errors='replace')
            return raw_text.replace(u'\x80', u' ').strip().replace("\n", " ").replace("\t", " ")
    return None

In [82]:
with codecs.open(file_check_single, 'r', encoding='utf-8', errors='ignore') as f:
    readable_html = f.read()
    g_comment = get_comment(readable_html)
g_comment

"Je vends ma Triumph Tiger 1200 XRT rouge, toutes options.? État impeccable? Gravage? Mise en circulation le 5 juin 2018 (année modèle 2019).? Elle totalise 18264 Km au compteur garantis (mais évolutifs).? Entretien Triumph (factures des toutes les interventions)? Dernière révision 15514KM le 11 juillet 2020 avec changement des pneumatiques pour :120/70R19 DUNLOP TRAILSMART MAX FRONT à l'avant170/60R17 DUNLOP TRAILSMART MAX REAR à l'arrièrePlaquettes de freins neuves au 4 août 2020Aucun consommable à prévoir? Fiche technique--Trois cylindres, 1215 cm3, 11 CV fiscaux, 141 puissance réelleKeyless (pile neuve)Boîte 6 Vitesses avec indication du rapport engagéShifter Pro up and downFeux directionnels, warningFeux additionnels à LEDABS et antipatinage (DTC) optimisésSuspensions semi-actives avec réglage automatique selon la chargeRégulateur de vitesseÉcran TFT couleur paramétrableAffichage de la température et de l'heureModes de conduites : rain, road, sport, enduro et personnalisablesAide 

### get seller

In [83]:
def get_seller(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('p[class*="txtpresentation-vendeur"]')
    if len(step1)>0:
        if step1[0].text.find("particulier")>0:
            return ["Particulier", step1[0].text.split(":")[-1].strip().split("\n")[0].strip()]
        else:
            return ["Professionnel", step1[0].text.strip().split("\n")[0]]
    return None

In [84]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller = get_seller(readable_html)
g_seller

['Particulier', 'Martin M']

### Save image functions

In [117]:
def get_images(r, uniq_id):
    
    # get images url list
    image_list = []
    
    # get soup
    soup_ = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup_.find('div',id="listePhotos")
    
    if step1 != None:
        if len(step1)>0:
            step2 = step1.select("img")
            if len(step2)>0:
                if "src" in step2[0].attrs:
                    for k in range(len(step2)-1):
                        image_list.append(step2[0].attrs["src"].replace("_1.jpeg", "_" + str(k+1) + ".jpeg"))

    # save images
    k=1
    for image_url in image_list[0:3]:
        image_name = f'images/{uniq_id}-{k}.jpg'
        if os.path.isfile(image_name) is False:
            img_data = requests.get(image_url).content
            with open(image_name, 'wb') as handler:
                handler.write(img_data)
            try:
                image = Image.open(image_name) 
                ratio = image.size[0] / image.size[1]
                image = image.resize((300,int(300/ratio)))
                image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 50)
            except:
                pass
        k+=1
        
    return

In [ ]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_ref = get_reference(readable_html)
    get_images(readable_html, g_ref)

<div style="color:orangered; font-weight:700; font-size:20pt">TEST 100 ANNONCES</div>

In [ ]:
list_test = []
for single_ in os.listdir("pages")[0:100]:
    with open(f"pages/{single_}", 'r') as f:
        readable_html = f.read()
        g_ref = get_reference(readable_html)
        get_images(readable_html, g_ref)
        print(single_)

In [576]:
#list_test

## Extract 1 announce

In [119]:
def process_announce(file_name):
    """ this function add an announce content from paruvendu.fr to the dataframe """
    
    # load csv if exists or starting from template
    data_paruvendu = "../../../tresboncoin/data/paruvendu.csv"
    #
    if os.path.isfile(data_paruvendu) is False:
        df = announce_template.copy()
    else:
        data = pd.read_csv(data_paruvendu)
        df = announce_template.copy()
    
    # fill df with defined functions
    with open(f"annonces/{file_name}", 'r') as f:
        readable_html = f.read()
        
        df["url"] = get_url(readable_html)
        df["reference"] = get_reference(readable_html)
        df["unique id"] = get_uniq_id(get_url(readable_html))
        df["date_scrapped"] = datetime.datetime.now().strftime("%Y/%m/%d - %Hh%M")
        df["announce_publication_date"] = get_publication_date(readable_html)
        df["vehicle brand"] = get_brand(readable_html)
        df["vehicle type"] = get_type(readable_html)
        df["moto scoot"] = get_moto(readable_html)
        df["color"] = get_color(readable_html)
        df["vehicle condition"] = get_cond(readable_html)
        df["price"] = get_price(readable_html)
        df["city"] = get_city(readable_html)
        df["postal code"] = get_postalcode(readable_html)
        df["vehicle release date"] = get_releasedate(readable_html)
        df["mileage"] = get_mileage(readable_html)
        df["Fiscal power [HP]"] = get_power(readable_html)
        df["engine capacity [CC]"] = get_capa(readable_html)
        df["comments"] = get_comment(readable_html)
        df["seller"] = get_seller(readable_html)[0]
        df["seller_name"] = get_seller(readable_html)[1]
    
        # save images
        get_images(readable_html, get_reference(readable_html))
        
    # concatenate to csv and write
    try:
        data = pd.concat([data, df], axis=0)
        data.to_csv(path_or_buf = data_paruvendu, index=False)
    except:
        df.to_csv(path_or_buf = data_paruvendu, index=False)
    
    # deplacer le fichier html traité dans le vault
    subprocess.run(["mv", f"annonces/{file_name}", "vault"])
    return

In [124]:
for single_ in os.listdir("annonces"):
    process_announce(single_)

##### test

In [3]:
data_paruvendu = "../../../tresboncoin/data/paruvendu.csv"
data = pd.read_csv(data_paruvendu)
data.tail(10)

,url,reference,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,moto scoot,color,vehicle condition,price,city,postal code,vehicle release date,mileage,Fiscal power [HP],engine capacity [CC],comments,seller,seller_name
1400,https://www.paruvendu.fr/a/moto-scooter/moto/h...,36482,1252884494,2021/06/05 - 15h46,04/06/2021,HONDA,NaN,MOTO,NaN,NaN,2500.0,Sète,34200,2005.0,2340.0,NaN,50.0,"Pour toute information, contactez un de nos co...",Professionnel,BONNIE AND CAR
1401,https://www.paruvendu.fr/a/moto-scooter/moto/h...,WV166372520,1250096068,2021/06/05 - 15h46,03/06/2021,HONDA,Roadster,MOTO,Gris,Bon,1750.0,Perros-Guirec,22700,2003.0,70000.0,77.0,600.0,"HONDA, 600 Cm3 , Roadster, Essence 4 temps, 20...",Particulier,Et G
1402,https://www.paruvendu.fr/a/moto-scooter/moto/b...,WV167347286,1253586439,2021/06/05 - 15h46,04/06/2021,BMW,Routière,MOTO,Rouge,Bon,2800.0,Achères,78260,2001.0,91500.0,11.0,1200.0,"BMW, 1200 Cm3 , Routière, Essence 4 temps, 200...",Particulier,Jacko
1403,https://www.paruvendu.fr/a/moto-scooter/moto/y...,38657,1254486123,2021/06/05 - 15h46,03/06/2021,YAMAHA,NaN,MOTO,NaN,NaN,4190.0,Soisy-sur-Seine,91450,2012.0,17870.0,NaN,600.0,"Pour toute information, contactez un de nos co...",Professionnel,BONNIE AND CAR
1404,https://www.paruvendu.fr/a/moto-scooter/moto/b...,WV167296789,1253412238,2021/06/05 - 15h46,04/06/2021,BMW,Routière,MOTO,NaN,NaN,3700.0,Saint-Jean-du-Falga,09100,2004.0,105000.0,95.0,1150.0,"BMW, 1150 Cm3 , Routière, Essence 4 temps, 200...",Particulier,Georges R
1405,https://www.paruvendu.fr/a/moto-scooter/moto/h...,WV167607875,1254347020,2021/06/05 - 15h46,04/06/2021,HONDA,Roadster,MOTO,NaN,NaN,3000.0,Sombernon,21540,1995.0,33000.0,7.0,650.0,"HONDA, 650 Cm3 , Roadster, Essence 4 temps, 19...",Particulier,Bunuel
1406,https://www.paruvendu.fr/a/moto-scooter/moto/t...,WV167705434,1254709325,2021/06/05 - 15h46,05/06/2021,TRIUMPH,Trail,MOTO,NaN,NaN,11000.0,Belpech,11420,2018.0,23000.0,10.0,1050.0,"TRIUMPH, 1050 Cm3 , Trail, 2018, 23000 Km , 11...",Particulier,Gilles A
1407,https://www.paruvendu.fr/a/moto-scooter/moto/b...,2050401,1254742196,2021/06/05 - 15h46,05/06/2021,BMW,Routière,MOTO,NaN,NaN,19990.0,Billy-Montigny,62420,2020.0,8065.0,NaN,1250.0,BMW R 1250 RT BMW R 1250 RT ETAT PROCHE DU NEU...,Professionnel,DELERUE L'EXPERIENCE MOTO
1408,https://www.paruvendu.fr/a/moto-scooter/moto/h...,WV167712227,1254745603,2021/06/05 - 15h46,05/06/2021,HARLEY-DAVIDSON,Routière,MOTO,Noir,Irréprochable,6900.0,Fontaine-lès-Luxeuil,70800,2009.0,17700.0,9.0,883.0,"HARLEY-DAVIDSON, 83 Cm3 , Roadster, 17700 Km ,...",Particulier,Caroline T
1409,https://www.paruvendu.fr/a/moto-scooter/moto/s...,WV167711718,1254744235,2021/06/05 - 15h46,05/06/2021,SUZUKI,Sportive,MOTO,NaN,NaN,1000.0,Graçay,18310,2019.0,517.0,50.0,50.0,"Bonjour, je vend ma moto Rg 50 Wolf de 1991 , ...",Particulier,Gwendoline G


In [183]:
data.shape

(1220, 20)

In [184]:
100 * data.isnull().sum().sort_values(ascending=False)/len(data)

color                        88.442623
vehicle condition            88.196721
Fiscal power [HP]            20.901639
vehicle type                 17.213115
mileage                       2.377049
engine capacity [CC]          2.131148
vehicle release date          1.229508
price                         0.655738
comments                      0.163934
announce_publication_date     0.163934
city                          0.081967
postal code                   0.081967
seller                        0.000000
url                           0.000000
reference                     0.000000
moto scoot                    0.000000
vehicle brand                 0.000000
date_scrapped                 0.000000
unique id                     0.000000
seller_name                   0.000000
dtype: float64

In [185]:
data.iloc[-1]["url"]

'https://www.paruvendu.fr/a/moto-scooter/moto/ducati/1200-cm3/1254636763A1KVMOMODU'

In [186]:
1251175264 in list(data["unique id"])

True